In [ ]:
import matplotlib
!pip install transformers==4.12.5
!pip install datasets
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION


from datasets import load_dataset
from transformers import (AutoTokenizer, AutoConfig,
                          AutoModelForSequenceClassification, TrainingArguments, Trainer)

#from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import torch


def load_dataset_new():
    file1 = open('/content/drive/My Drive/twitter-datasets/train_pos_full.txt', 'r')
    reviewspos = file1.readlines()
    file1 = open('/content/drive/My Drive/twitter-datasets/train_neg_full.txt', 'r')
    reviewsneg = file1.readlines()
    return reviewspos, reviewsneg

import re
def pre_process(text):
  text = re.sub(r"<user>", "",text)#remove <user>

  others = '1234567890' #remove these symbols
  for p in others:
        text = text.replace(p,'')
  return text



In [ ]:
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
reviewspos, reviewsneg = load_dataset_new()
reviews = reviewspos + reviewsneg
pos = [3] * 1250000
neg = [1] * 1250000
labels = pos + neg


In [ ]:
import pandas as pd

dtrain = {'text': reviews, 'labels': labels}
df = pd.DataFrame(data=dtrain)
df['text'] = df['text'].apply(lambda x: pre_process(x))

df.to_pickle("train_full.pkl")



In [ ]:
#dtest = {'text': val_texts, 'labels': val_labels}
#df = pd.DataFrame(data=dtest)
dfrand = df.sample(n=200000, random_state=99502)
dfrand.to_pickle("test_full.pkl")

In [ ]:
datasetall = load_dataset("pandas", data_files={'train':"/content/train_full.pkl",'test':"/content/test_full.pkl" })


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-df2823b124f34f4b/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasetall


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2500000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 200000
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', use_fast=True)
def tokenize_function(x):
    return tokenizer(x["text"], max_length=128, padding="max_length", truncation=True)
tokenized_datasets = datasetall.map(tokenize_function, batched=True)

  0%|          | 0/2500 [00:01<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [ ]:
# !pip install torch==1.10.0

In [ ]:

  
train_dataset = tokenized_datasets["train"].shuffle(seed=907)
eval_dataset = tokenized_datasets["test"].shuffle(seed=907)

modelconfig = AutoConfig.from_pretrained(pretrained_model_name_or_path='cardiffnlp/twitter-xlm-roberta-base-sentiment', num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-xlm-roberta-base-sentiment', config=modelconfig)
#model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-84375/')


#model.to(device)
#dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
model.train()


WRAPPED_MODEL = xmp.MpModelWrapper(model)


#total_steps = len(dataloader) * 4
#scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps=0, t_total= total_steps)


Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #_, _, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        #'macro f1': macro_f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
#trainer = Trainer(
#    model=model, args=TrainingArguments("test_trainer", evaluation_strategy="epoch",save_strategy ="epoch" ,tpu_num_cores=8,
#        load_best_model_at_end=True),compute_metrics=compute_metrics, train_dataset=train_dataset, eval_dataset=eval_dataset
#)

In [ ]:

def _mp_fn(index):
    device = xm.xla_device()
    # We wrap this 
    model = WRAPPED_MODEL.to(device)
    trainer = Trainer(
    model=model, args=TrainingArguments(output_dir = '/content/drive/MyDrive/twitter-datasets/modelssave', evaluation_strategy="steps",eval_steps=9375,save_steps=9375,load_best_model_at_end=True), compute_metrics=compute_metrics, train_dataset=train_dataset, eval_dataset=eval_dataset)
    trainer.place_model_on_device = False

    trainer.train()

xmp.spawn(_mp_fn, start_method="fork")

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 2500000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 117189


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200000
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and b

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


...

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
9375,0.000000,0.000000,0.502620,0.334496,0.251310,0.500000


Configuration saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-9375/config.json
Model weights saved in /content/drive/MyDrive/twitter-datasets/modelssave/checkpoint-9375/pytorch_model.bin


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("cardiff-xlm-roberta-base",use_temp_dir=True )
tokenizer.push_to_hub("ç",use_temp_dir=True)